In [3]:
import gymnasium as gym
import numpy as np

try:
     del gym.envs.registry['lungs-v0']
except:
     pass

gym.envs.register(
     id='lungs-v0',
     entry_point='simulator.envs.lung.lungs:LungEnv',
     max_episode_steps=250,
)

G = 0.5
LARGE_MAZE_DIVERSE_G = [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                        [1, G, 0, 0, 0, 1, G, 0, 0, 0, 0, 1],
                        [1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1],
                        [1, 0, 0, 0, 0, G, 0, 1, 0, 0, G, 1],
                        [1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1],
                        [1, 0, G, 1, 0, 1, 0, 0, 0, 0, 0, 1],
                        [1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1],
                        [1, 0, 0, 1, G, 0, G, 1, 0, G, 0, 1],
                        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]



env = gym.make('lungs-v0', maze_map = LARGE_MAZE_DIVERSE_G, max_episode_steps=250)


/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment lungs-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchSlide-v1 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchSlide-v2 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/envs/registration.py:694: UserWarning: WARN: Overriding environment FetchPickAndPlace-v1 already in registry.
  logger.warn(f"Overriding environ

In [4]:
# Box(4,) means that it is a Vector with 4 components
print("Observation space:", env.observation_space)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

Observation space: Dict('achieved_goal': Box(-inf, inf, (2,), float64), 'desired_goal': Box(-inf, inf, (2,), float64), 'maze_map': Box(-9223372036854775806, 9223372036854775805, (108,), int64), 'observation': Box(-inf, inf, (4,), float64))
Action space: Box(-1.0, 1.0, (2,), float32)


# Train

In [5]:
from stable_baselines3 import PPO

model = PPO("MultiInputPolicy", env, verbose=0, device='cpu')
model.learn(total_timesteps=1000000, progress_bar=True)


vec_env = model.get_env()
obs = vec_env.reset()
for i in range(10):
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, done, info = vec_env.step(action)
    vec_env.render()
    # VecEnv resets automatically
    # if done:
    #   obs = env.reset()

env.close()

/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:135: 
UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be int64, actual 
type: float64
  logger.warn(

/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:159: 
UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")

/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be int64, actual type: float64
  logger.warn(
/home/flaccagora/.miniconda3/envs/RoboSurg/lib/python3.9/site-packages/gymnasium/utils/passive_env_checker.py:159: UserWarning: WARN: The obs returned by the `reset()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


KeyboardInterrupt: 

# Save and/or Test

In [4]:
model.save("ppo_lungs_1M")
del model
del env

NameError: name 'model' is not defined

In [2]:
from stable_baselines3 import PPO
model = PPO.load("ppo_lungs_1M")
env = gym.make('lungs-v0', render_mode = 'human',maze_map = LARGE_MAZE_DIVERSE_G, max_episode_steps=250)

In [3]:
observation, info = env.reset(seed=412)

terminated = False
truncated = False

while True:
    while not terminated and not truncated:
        #action = policy(observation)  # custom policy
        #action = env.action_space.sample()  #
        action, _states = model.predict(observation, deterministic=True)
        
        observation, reward, terminated, truncated, info = env.step(action)

    observation, info = env.reset()
    terminated = False
    truncated = False
env.close()

: 